# TODO:


1.   Finish DataLoaders so you don't have to do manual batch and/or SGD
2.   Try other models like attention models etc.
3.   Use BERT or other SOTA models
4.   Reprocess Dataset uinsg torchtext


Download Existing Repo


In [0]:
!wget https://github.com/lohzhunyewcs/Toxic-Behaviour-Detector/archive/master.zip

--2020-04-06 12:51:33--  https://github.com/lohzhunyewcs/Toxic-Behaviour-Detector/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lohzhunyewcs/Toxic-Behaviour-Detector/zip/master [following]
--2020-04-06 12:51:34--  https://codeload.github.com/lohzhunyewcs/Toxic-Behaviour-Detector/zip/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ]   7.00K  --.-KB/s    in 0.001s  

2020-04-06 12:51:35 (11.9 MB/s) - ‘master.zip’ saved [7167]



In [0]:
!ls

master.zip  sample_data


In [0]:
!unzip -q master.zip

In [0]:
import os
print(os.listdir())
os.rename('Toxic-Behaviour-Detector-master', 'src')

['.config', 'master.zip', 'Toxic-Behaviour-Detector-master', 'sample_data']


Downloading dataset

In [0]:
os.makedirs('src/labeled_data', exist_ok=True)

In [0]:
import requests

def download_file_from_google_drive(id, destination):
    def get_confirm_token(response):
        for key, value in response.cookies.items():
            if key.startswith('download_warning'):
                return value

        return None

    def save_response_content(response, destination):
        CHUNK_SIZE = 32768

        with open(destination, "wb") as f:
            for chunk in response.iter_content(CHUNK_SIZE):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    


if __name__ == "__main__":
    # TAKE ID FROM SHAREABLE LINK
    file_id = '1rqC8VZskMf1Shgo6ejTu5nnbZn4K5ngT'
    # DESTINATION FILE ON YOUR DISK
    destination = 'src/labeled_data/labeled_data.csv'
    download_file_from_google_drive(file_id, destination)

We'll be using 3 Libraries to train
1.   TensorFlow
2.   PyTorch
3.   PyTorch-Lightning(for their seamless TPU support)


TensorFlow

PyTorch

In [0]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.utils.data.dataset import random_split


In [0]:
class DotaChatDataset(Dataset):
  def __init__(self, file_name):
    self.chats = []
    self.is_toxics = []
    index_counter = 0
    self.word_to_idx = {}
    self.max_length = 0
    with open(f'src/labeled_data/{file_name}', 'r') as file:
      for index, line in enumerate(file):
        if index == 0:
          continue
        line = line.strip().split(',')
        chat, is_toxic = line
        self.chats.append(chat)
        self.is_toxics.append(int(is_toxic))

        splitted_chat = chat.split(' ')
        self.max_length = max(self.max_length, len(splitted_chat))
        for word in splitted_chat:
          if word not in self.word_to_idx:
            self.word_to_idx[word] = index_counter
            index_counter += 1
    self.offsets = [0] + [len(i) for i in self.chats]
  
  def __len__(self):
    return len(self.chats)

  def __getitem__(self, idx):
    x = [self.word_to_idx[i] for i in self.chats[idx].split(' ')]
    x = torch.tensor(x)
    # x = torch.cat(x)
    return {
        'x': x.float(),
        'y': torch.tensor([self.is_toxics[idx]]).float()
    }



In [0]:
chat_dataset = DotaChatDataset('labeled_data.csv')

In [0]:
print(len(chat_dataset.word_to_idx))

97197


In [0]:
# Test if dataset works
for i in range(5): 
  print(chat_dataset[i])
  a = chat_dataset[i]
  print(type(a['y']))
  break

{'x': tensor([0.]), 'y': tensor([0.])}
<class 'torch.Tensor'>


Model

In [0]:
class IsToxic(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = None

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return torch.sigmoid(predictions[-1])

In [0]:
VOCAB_SIZE = len(chat_dataset.word_to_idx)
EMBED_DIM = 32
NUM_CLASS = 1
model = IsToxic()

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
def my_round(item):
  if item >= 0.5:
    return 1
  else:
    return 0

# Training Function
def train(train_dataset):
  train_loss = 0
  train_acc = 0
  
  for index, data in enumerate(train_dataset):
    optimizer.zero_grad()
    model.hidden_cell = None
    x = data['x'].to(device)
    y = data['y'].to(device)
    output = model(x)
    # print(f'x: {x} output: {output}, y: {y}')
    loss = criterion(output, y)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    # print(f'output: {output}')
    train_acc += int(my_round(output.item()) == y)
  
  return train_loss/(index + 1), train_acc/(index + 1)

# validation Function
def valid(val_dataset):
  valid_loss = 0
  valid_acc = 0
  
  for index, data in enumerate(val_dataset):
    x = data['x'].to(device)
    y = data['y'].to(device)
    with torch.no_grad():
      output = model(x)
      loss = criterion(output, y)
      valid_loss += loss.item()
      valid_acc += int(my_round(output.item()) == y)
  
  return valid_loss/(index + 1), valid_acc/(index + 1)

In [0]:

model = model.to(device)
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.00)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)
EPOCHS = 5

train_size = int(len(chat_dataset) * 0.95)
train_dataset, val_dataset = random_split(chat_dataset, [train_size, len(chat_dataset) - train_size])

for epoch in range(EPOCHS):
  start_time = time.time()
  train_loss, train_acc = train(train_dataset)
  valid_loss, valid_acc = valid(val_dataset)


  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 17 minutes, 26 seconds
	Loss: 15.4012(train)	|	Acc: 47.5%(train)
	Loss: 14.8848(valid)	|	Acc: 46.1%(valid)
Epoch: 2  | time in 17 minutes, 33 seconds
	Loss: 14.7898(train)	|	Acc: 46.5%(train)
	Loss: 14.8844(valid)	|	Acc: 46.1%(valid)
Epoch: 3  | time in 17 minutes, 32 seconds
	Loss: 14.7898(train)	|	Acc: 46.5%(train)
	Loss: 14.8844(valid)	|	Acc: 46.1%(valid)
Epoch: 4  | time in 17 minutes, 36 seconds
	Loss: 14.7898(train)	|	Acc: 46.5%(train)
	Loss: 14.8844(valid)	|	Acc: 46.1%(valid)
Epoch: 5  | time in 17 minutes, 41 seconds
	Loss: 14.7898(train)	|	Acc: 46.5%(train)
	Loss: 14.8844(valid)	|	Acc: 46.1%(valid)


PyTorchLighNing

In [0]:
!pip install pytorch-lightning

In [0]:
import pytorch_lightning